In [1]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob
import random
import os

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.9/605.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
ROOT_DIR = '/content/gdrive/MyDrive/ColabNotebooks/Data'

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
def plot_box(image, bboxes, labels):
    # Need the image height and width to denormalize
    # the bounding box coordinates
    h, w, _ = image.shape
    for box_num, box in enumerate(bboxes):
        x1, y1, x2, y2 = yolo2bbox(box)
        # Denormalize the coordinates.
        xmin = int(x1*w)
        ymin = int(y1*h)
        xmax = int(x2*w)
        ymax = int(y2*h)

        thickness = max(2, int(w/275))

        cv2.rectangle(
            image,
            (xmin, ymin), (xmax, ymax),
            color=(0, 0, 255),
            thickness=thickness
        )
    return image

In [ ]:
# Function to plot images with the bounding boxes.
def plot(image_paths, label_paths, num_samples):
    all_images = []
    all_images.extend(glob.glob(image_paths+'/*.jpg'))
    all_images.extend(glob.glob(image_paths+'/*.JPG'))

    all_images.sort()

    num_images = len(all_images)

    plt.figure(figsize=(15, 12))
    for i in range(num_samples):
        j = random.randint(0,num_images-1)
        image_name = all_images[j]
        image_name = '.'.join(image_name.split(os.path.sep)[-1].split('.')[:-1])
        image = cv2.imread(all_images[j])
        with open(os.path.join(label_paths, image_name+'.txt'), 'r') as f:
            bboxes = []
            labels = []
            label_lines = f.readlines()
            for label_line in label_lines:
              #try:
                label = label_line[0]
                bbox_string = label_line[2:]
                bbox_string = bbox_string.strip(' ')
                x_c, y_c, w, h = bbox_string.split(' ')
                x_c = float(x_c)
                y_c = float(y_c)
                w = float(w)
                h = float(h)
                bboxes.append([x_c, y_c, w, h])
                labels.append(label)
                #print("#Data:", bbox_string)
                #print("#File name:", image_name+'.txt')

             # except ValueError:
              #  print("Data:", bbox_string, len(bbox_string.split(' ')))
               # for a, i in enumerate(bbox_string.split(' ')):
                 # print(a, i)
                #print("File name:", image_name+'.txt')
                #break

        result_image = plot_box(image, bboxes, labels)
        plt.subplot(num_samples//4, 4, i+1)
        plt.imshow(result_image[:, :, ::-1])
        plt.axis('off')
    plt.subplots_adjust(wspace=1)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize a few training images.
plot(
    image_paths=f'{ROOT_DIR}/train/images/',
    label_paths=f'{ROOT_DIR}/train/labels/',
    num_samples=4,
)

In [ ]:
# Sample training for 5 epoch.
EPOCHS = 100
#model = YOLO("litter_v8.yaml")
!yolo task='detect' \
  mode=train \
  model=runs/detect/yolov8n_v8_50e6/weights/best.pt \
  imgsz=416 \
  data='/content/data.yaml' \
  epochs={EPOCHS} \
  batch=8 \
  name=yolov8n_v8_50e

In [ ]:
from IPython.display import display, Image

In [ ]:
Image(filename=f'runs/detect/yolov8n_v8_50e7/confusion_matrix.png', width=600)

In [ ]:
Image(filename=f'runs/detect/yolov8n_v8_50e6/results.png', width=600)

In [ ]:
Image(filename=f'runs/detect/yolov8n_v8_50e7/results.png', width=600)

In [ ]:
!yolo task=detect mode=val model=runs/detect/yolov8n_v8_50e7/weights/best.pt name=yolov8n_eval data=data.yaml

In [ ]:
!yolo task=detect \
mode=predict \
model=runs/detect/yolov8n_v8_50e7/weights/best.pt \
source='/content/gdrive/MyDrive/ColabNotebooks/Pothole_Data/test/images/' \
imgsz=416 \
name=yolov8n_v8_50e_infer416 \
hide_labels=False

In [ ]:
# Plot and visualize images in a 2x2 grid.
def visualize(result_dir, num_samples=4):
    """
    Function accepts a list of images and plots
    them in a 2x2 grid.
    """
    plt.figure(figsize=(30, 12))
    image_names = glob.glob(os.path.join(result_dir, '*.jpg'))
    random.shuffle(image_names)
    for i, image_name in enumerate(image_names):
        image = plt.imread(image_name)
        plt.subplot(num_samples//2, 2, i+1)
        plt.imshow(image)
        plt.axis('off')
        if i == num_samples-1:
            break
    plt.tight_layout()
    plt.show()

In [ ]:
visualize('runs/detect/yolov8n_v8_50e_infer4163/', 12)

In [ ]:
!yolo task=detect \
mode=predict \
model=runs/detect/yolov8n_v8_50e6/weights/best.pt \
source='/content/video/Video002.mp4' \
imgsz=416 \
name=yolov8n_v8_50e_infervideo \
hide_labels=True

In [ ]:
from moviepy.editor import *

path="runs/detect/yolov8n_v8_50e_infervideo2/Video002.avi"

clip=VideoFileClip(path)
clip.ipython_display(width=280)

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os

# Input video path
save_path = "/content/__temp__.mp4"

# Compressed video path
compressed_path = "/content/__temp__compressed.mp4"

os.system(f"ffmpeg -i {save_path} -vcodec libx264 {compressed_path}")

# Show video
mp4 = open(compressed_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
!zip -r runs_yolo.zip runs/

In [ ]:
from google.colab import files
files.download('runs_yolo.zip')